In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
!pip install -q -U git+https://github.com/sbrugman/SDGym.git@v0.2.2-hw

In [ ]:
!pip install -q -U ../../

In [2]:
from timeit import default_timer as timer
from functools import partial
from random import choices
import logging

In [3]:
import sdgym
from sdgym import load_dataset
from sdgym import benchmark
from sdgym import load_dataset

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx

In [5]:
import pgmpy

from pgmpy.models import BayesianModel
from pgmpy.estimators import TreeSearch, HillClimbSearch, BicScore, ExhaustiveSearch, BayesianEstimator
from pgmpy.sampling import BayesianModelSampling

In [6]:
import xgboost as xgb
from xgboost import XGBClassifier

In [7]:
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.isotonic import IsotonicRegression

In [8]:
from scipy import interpolate

In [9]:
from synthsonic.models.kde_utils import kde_smooth_peaks_1dim, kde_smooth_peaks
from synthsonic.models.kde_copula_nn_pdf import KDECopulaNNPdf

In [10]:
import matplotlib.pyplot as plt
%matplotlib inline

In [11]:
logging.basicConfig(level=logging.INFO)

In [12]:
dataset_name = 'census'

In [13]:
data, categorical_columns, ordinal_columns = load_dataset(dataset_name)

INFO:sdgym.data:Loading dataset census (all variables, -1 distinct values threshold)
INFO:sdgym.data:Columns to zero-code: []


In [14]:
data.shape

(199523, 41)

In [15]:
categorical_columns, ordinal_columns

([1,
  2,
  3,
  4,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  39,
  40],
 [])

In [16]:
for i in range(data.shape[1]):
    print (i, len(np.unique(data[:, i])))

0 91
1 9
2 52
3 47
4 17
5 1240
6 3
7 7
8 24
9 15
10 5
11 10
12 2
13 3
14 6
15 8
16 132
17 113
18 1478
19 6
20 6
21 51
22 38
23 8
24 10
25 9
26 10
27 3
28 4
29 7
30 5
31 43
32 43
33 43
34 5
35 3
36 3
37 3
38 53
39 2
40 2


# run sdgym

In [27]:
def KDECopulaNNPdf_Synthesizer(real_data, categorical_columns, ordinal_columns):
    all_features = list(range(real_data.shape[1]))
    numerical_features = list(set(all_features) - set(categorical_columns + ordinal_columns))
    data = np.float64(real_data)
    
    n_samples = data.shape[0]
    n_features = data.shape[1]
    
    clf = xgb.XGBClassifier(
        n_estimators=250,
        reg_lambda=1,
        gamma=0,
        max_depth=9
    )
#     clf = MLPClassifier(alpha=0.1, random_state=0, max_iter=1000, early_stopping=True)
    
    kde = KDECopulaNNPdf(
        use_KDE=False, 
        categorical_columns=categorical_columns+ordinal_columns,
        distinct_threshold=-1,
        n_bins=25,
#        clf=clf,
#         ordering='mi',
    )
    kde = kde.fit(data)
    
#     X_gen, sample_weight = kde.sample(n_samples)
    X_gen = kde.sample_no_weights(n_samples, show_progress=True, mode='cheap')
    
    X_gen[:, categorical_columns+ordinal_columns] = np.round(X_gen[:, categorical_columns+ordinal_columns])
    X_gen = np.float32(X_gen)
    
    print(X_gen[:, :5])

    return X_gen

In [28]:
def KDECopulaNNPdf_SynthesizerInteger(real_data, categorical_columns, ordinal_columns):
    """Census has integer only..."""
    data = KDECopulaNNPdf_Synthesizer(real_data, categorical_columns, ordinal_columns)
    data = np.round(data)
    
    print(data[:, :5])

    return data

In [29]:
from sdgym.synthesizers import (
    CLBNSynthesizer, CTGANSynthesizer, IdentitySynthesizer, IndependentSynthesizer,
    MedganSynthesizer, PrivBNSynthesizer, TableganSynthesizer, TVAESynthesizer,
    UniformSynthesizer, VEEGANSynthesizer)

all_synthesizers = [
    #IdentitySynthesizer,
    #IndependentSynthesizer,
    #KDECopulaNNPdf_Synthesizer,
    KDECopulaNNPdf_SynthesizerInteger,
]

In [30]:
scores = sdgym.run(synthesizers=all_synthesizers, datasets=[dataset_name], iterations=1)

INFO:sdgym.benchmark:0%|          | 0/1 [00:00<?, ?it/s]
INFO:sdgym.benchmark:Evaluating KDECopulaNNPdf_SynthesizerInteger on dataset census; iteration 0; 1.06 GB
INFO:sdgym.data:Loading dataset census (all variables, -1 distinct values threshold)
INFO:sdgym.data:Columns to zero-code: []
INFO:sdgym.benchmark:Running KDECopulaNNPdf_SynthesizerInteger on dataset census; iteration 0; 1.06 GB
INFO:KDECopulaNNPdf:Processing 7 numerical and 34 categorical columns
INFO:KDECopulaNNPdf:Transforming numerical variables.
INFO:KDECopulaNNPdf:Configuring Bayesian Network (cat+num).
INFO:KDECopulaNNPdf:Configuring classifier.
INFO:KDECopulaNNPdf:Generating 250000 data points.

Generating for node: 36: 100%|██████████| 41/41 [00:09<00:00,  4.13it/s]
INFO:KDECopulaNNPdf:Fitting discriminative learner: selected 41 features.
INFO:KDECopulaNNPdf:Calibrating classifier.
INFO:KDECopulaNNPdf:Maximum weight found: 176.837778248887
INFO:KDECopulaNNPdf:Generating 199523 data points.

Generating for node: 15:  

Generating for node: 36: 100%|██████████| 41/41 [00:07<00:00,  5.20it/s]
INFO:sdgym.benchmark:Scoring KDECopulaNNPdf_SynthesizerInteger on dataset census; iteration 0; 618.84 MB


[[35.999775  3.        2.        2.        3.      ]
 [38.000553  4.       31.       10.       15.      ]
 [68.99781   6.        6.       35.        0.      ]
 ...
 [82.602974  0.        0.        0.        0.      ]
 [39.001225  1.       24.        1.        2.      ]
 [48.002174  1.       24.        6.        0.      ]]
[[36.  3.  2.  2.  3.]
 [38.  4. 31. 10. 15.]
 [69.  6.  6. 35.  0.]
 ...
 [83.  0.  0.  0.  0.]
 [39.  1. 24.  1.  2.]
 [48.  1. 24.  6.  0.]]


INFO:sdgym.evaluate:Evaluating using binary classifier DecisionTreeClassifier
INFO:sdgym.evaluate:Evaluating using binary classifier AdaBoostClassifier
INFO:sdgym.evaluate:Evaluating using binary classifier LogisticRegression
INFO:sdgym.evaluate:Evaluating using binary classifier MLPClassifier
/Users/maxbaak/opt/miniconda3/envs/synthsonic/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  warnings.warn(
INFO:sdgym.benchmark:Finished KDECopulaNNPdf_SynthesizerInteger on dataset census; iteration 0; 788 MB
INFO:sdgym.benchmark:100%|##########| 1/1 [04:06<00:00, 246.60s/it]


In [31]:
scores

,census/accuracy,census/f1,timestamp
VEEGANSynthesizer,0.764352,0.051816,2020-04-12 09:41:35.096775
CLBNSynthesizer,0.902265,0.285714,2020-10-17 09:46:54.494331
CTGAN,0.923290,0.328815,2020-10-17 09:46:54.494331
CTGANSynthesizer,0.904780,0.378843,2020-10-17 09:46:54.494331
CopulaGAN,0.890427,0.387663,2020-10-17 09:46:54.494331
GaussianCopulaCategorical,0.937647,0.000000,2020-10-17 09:46:54.494331
GaussianCopulaCategoricalFuzzy,0.791698,0.053528,2020-10-17 09:46:54.494331
GaussianCopulaOneHot,0.934770,0.132830,2020-10-17 09:46:54.494331
IdentitySynthesizer,0.905330,0.463875,2020-10-17 09:46:54.494331
IndependentSynthesizer,0.664293,0.060412,2020-10-17 09:46:54.494331


In [32]:
scores.tail(4)

,census/accuracy,census/f1,timestamp
TVAESynthesizer,0.934452,0.382321,2020-10-17 09:46:54.494331
TableganSynthesizer,0.936630,0.272120,2020-10-17 09:46:54.494331
UniformSynthesizer,0.541408,0.108323,2020-10-17 09:46:54.494331
KDECopulaNNPdf_SynthesizerInteger,0.901800,0.390823,2021-05-13 20:54:09.467966
